In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
import string
import pandas as pd
import numpy as np

#download assets from nltk
#nltk.download('stopwords')
#nltk.download('punkt')

def tfidf(corpus):
    '''
    Computes the TF-IDF (term frequency - inverse document frequency) matrix

    Args
    - corpus: a list of sentences (documents) that need to be summarized

    Returns
    - tfidfVec: an m x n matrix of the corpus. m = number of different terms used in the documents, n = number of documents 
    - vocab: all the unique words used in the corpus, excluding stop words
    '''

    vectorizer = TfidfVectorizer(stop_words = stopwords.words('english'))
    #vectorizer = CountVectorizer(stop_words='english')
    tfidfVec = vectorizer.fit_transform(corpus)
    vocab = vectorizer.get_feature_names()
        
    return tfidfVec, vocab

def svd(tfidfVec):
    '''
    Gives the singular value decomposition of an m x n matrix.
    A = U * sigma * V^t
    
    Args
    - tfidfVec: an m x n matrix. m = number of documents or sentences, n = number of terms

    Returns
    - U: an m x r matrix of left singular values (document-topic table). r = number of topics
    - sigma: an r x r diagonal matrix of singular values in decreasing order across the diagonal
    - V^t: an n x r matrix of right singular values (term-topic table)
    '''

    lsa = TruncatedSVD(n_components = 10, n_iter=20)
    u = lsa.fit_transform(tfidfVec)
    sigma = lsa.singular_values_
    vt = lsa.components_.T

    return u, sigma, vt

def getImportantSentences(u, sigma):
    '''
    Uses the LSA enhancement described by Josef Steinberg, et al.
    Take all topics that have singular values > half of the largest singular value

    Compute sk = sqrt(sum(v_ki^2 * sigma_i^2) from i = 1 to n)
    sk is the length of the vector of the kth sentence
    n is the number of topics 

    Args
    - U, sigma matrices from SVD

    Returns
    - Vector of indices corresponding to the sentences in corpus sorted in descending order
      of importance
    '''

    #look for the sigma value range that we need to consider using binary search
    #sigma array is sorted in descending order and will never be empty
    l, r, target = 0, len(sigma), sigma[0]/2
    while l < r:
        mid = l + (r-l)//2

        if sigma[mid] < target:
            r = mid
        else:
            l = mid + 1
    sigmaBound = l

    uSlice = u[:, :sigmaBound]
    sigmaSlice = sigma[:sigmaBound]
    uSq = np.square(uSlice)
    sigSq = np.square(np.diag(sigmaSlice))
    prod = np.matmul(uSq, sigSq)
    result = np.sqrt(np.sum(prod, axis = 1)).T

    return (-result).argsort()

def createWordToSentenceMap(corpus):
    '''
    Creates a dictionary that maps a word from the vocab to all sentences with that word in the corpus.

    Args
    - corpus of sentences used in this summary

    Returns
    - the dictionary described
    '''
    
    wordToSentence = {}
    stopWords = set(stopwords.words('english'))

    for i, doc in enumerate(corpus):
        #remove punctuation while preserving contractions in text
        sanitizeText = doc.translate(str.maketrans('', '', string.punctuation))
        tokenized = word_tokenize(sanitizeText)
        #remove duplicate words
        tokenized = list(set([word.lower() for word in tokenized]))

        for word in tokenized:
            if word not in stopWords:
                if word not in wordToSentence:
                    wordToSentence[word] = [i]
                else:
                    wordToSentence[word].append(i)
    
    return wordToSentence

def extractSummary(u, sigma, k, corpus):
    '''
    Helper method to get the text summary.

    Summary will be taken from the top k sentences from getImportantSentences()
    for each topic.

    Args
    - U, sigma from SVD
    - k: number of sentences to include in summary
    - corpus: the list of sentences
    '''

    return [corpus[i] for i in getImportantSentences(u, sigma)[:k]]


In [2]:
def preProcess(blockText):
    '''
    Preprocesses the original text to be summarized by tokenizing the sentences and removing
    unnecessary characters.

    Args
    - blockText: text to be summarized

    Returns
    - list of sentences that can be used to create a summary
    '''

    tokenized = sent_tokenize(blockText) 
    return [token.replace('\n',' ') for token in tokenized]

In [3]:
text = u'''
The list of businesses impacted by a lockdown beginning Monday in Toronto and Peel Region were not clearly communicated, the owner of a Toronto massage spa says.

While the Ontario government offered a partial list of what would remain open after the COVID-19 shutdown begins at 12:01 a.m., Kate Armstrong, owner and director of Bahn Thai Spa, told the Star she was unsure whether her business would be impacted.

The Ontario government’s late-afternoon announcement on Friday stated that personal services, such as nail and hair salons, would now be closed. Missing, however, were details of all services included in the shutdown.

However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”

A spokesperson said that “under lockdown, regulated health professionals, including massage therapists, will be able to operate. Regulated health professionals such as registered massage therapists were not impacted and therefore not referenced.”

Working “in partnership with the chief medical officer of health and our local medical officers of health, we continue to closely monitor the evolving situation to advise if and when public health measures need to be adjusted,” the spokesperson also said.

In Ontario’s first lockdown last spring, physiotherapy, chiropractic services and massage therapists were among those to close their doors, which left some confused about what is happening this time around.

“We have to continue to communicate with clients that are calling and saying, ‘Are we seeing you on Monday or not?’ We’re having to say we’ll call you as soon as we know something more,” Armstrong said.

“It’s not like a haircut,” she said, adding that people are often seeking massage to treat physical pain or for mental health care.

To Armstrong, massage has been as important as mental healthcare for Ontarians during the months-long pandemic. “I see the fatigue setting in on everyone’s faces ... The stress is so high … right now, (with) people not being able to be with their families. It’s so important to have human touch.”

The Ontario Physiotherapy Association shared the news that physiotherapy services would be able to continue operations, said Shafiq Bhanji, president of Athlete’s Care Sports Medicine Centres.

“We received direction from our respective colleges and professional associations on Friday and over the weekend via email indicating that our services would not be impacted the upcoming lockdown,” Bhanji said in an email to the Star.

While Bhanji was able to confirm that Athlete’s Care could continue offering services and communicate that to clients via email, patients are still reaching out to verify whether they can keep their appointments.

“It seems there was a fair bit of confusion in the general public about whether or not these services would be impacted,” Bhanji said. “... We are fortunate that our colleges and professional associations acted quickly to inform their members.”
'''

corpus = preProcess(text)
display(corpus)
tfidfVec, vocab = tfidf(corpus)
wordToSentence = createWordToSentenceMap(corpus)
print(vocab)
print(wordToSentence)
print(tfidfVec)
print('----------------------------------------------------------')

u, sigma, vt = svd(tfidfVec)
numTopics = u.shape[1] + 1

dfSVD = pd.DataFrame(u, columns=[f'topic{str(i)}' for i in range(1, numTopics)])
docCol = pd.DataFrame({'Documents': corpus})
dfSVD = pd.concat([docCol, dfSVD], axis = 1)

display(dfSVD)
print('----------------------------------------------------------')
print(sigma)

print('----------------------------------------------------------')

dfVt = pd.DataFrame(vt, columns=[f'topic{str(i)}' for i in range(1, numTopics)])
vocabCol = pd.DataFrame({'Terms': vocab})
dfVt = pd.concat([vocabCol, dfVt], axis = 1)

display(dfVt)

for i in range(1, numTopics):
    dfVtSort = dfVt.sort_values(by=f'topic{i}', ascending=False)
    display(dfVtSort[['Terms', f'topic{i}']])
print('----------------------------------------------------------')


#df = pd.DataFrame()
#print(df)

[' The list of businesses impacted by a lockdown beginning Monday in Toronto and Peel Region were not clearly communicated, the owner of a Toronto massage spa says.',
 'While the Ontario government offered a partial list of what would remain open after the COVID-19 shutdown begins at 12:01 a.m., Kate Armstrong, owner and director of Bahn Thai Spa, told the Star she was unsure whether her business would be impacted.',
 'The Ontario government’s late-afternoon announcement on Friday stated that personal services, such as nail and hair salons, would now be closed.',
 'Missing, however, were details of all services included in the shutdown.',
 'However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”  A spokesperson said that “under lockdown, regulated health professionals, including massage th

['01', '12', '19', 'able', 'acted', 'adding', 'adjusted', 'advise', 'afternoon', 'also', 'among', 'announcement', 'appointments', 'armstrong', 'around', 'association', 'associations', 'athlete', 'bahn', 'beginning', 'begins', 'bhanji', 'bit', 'business', 'businesses', 'call', 'calling', 'care', 'centres', 'chief', 'chiropractic', 'clearly', 'clients', 'close', 'closed', 'closely', 'colleges', 'communicate', 'communicated', 'confirm', 'confirmed', 'confused', 'confusion', 'continue', 'could', 'covid', 'dentists', 'details', 'direction', 'director', 'doors', 'email', 'everyone', 'evolving', 'faces', 'fair', 'families', 'fatigue', 'first', 'fortunate', 'friday', 'general', 'government', 'hair', 'haircut', 'happening', 'health', 'healthcare', 'high', 'however', 'human', 'impacted', 'important', 'included', 'including', 'indicating', 'inform', 'kate', 'keep', 'know', 'last', 'late', 'left', 'like', 'list', 'local', 'lockdown', 'long', 'massage', 'measures', 'medical', 'medicine', 'members',

,Documents,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
0,The list of businesses impacted by a lockdown...,2.122050e-01,-5.689006e-02,-3.586889e-01,-2.520224e-01,4.926658e-01,-4.060439e-02,1.124230e-02,2.172241e-14,-4.564118e-01,2.536252e-01
1,While the Ontario government offered a partial...,3.251265e-01,-3.045537e-01,-4.457429e-01,-2.780163e-01,1.583704e-01,-6.354932e-02,2.703197e-01,-2.948216e-14,8.511480e-03,-1.706644e-02
2,The Ontario government’s late-afternoon announ...,2.498981e-01,-2.845500e-01,-3.005991e-01,-6.441886e-02,-2.800185e-01,1.884545e-01,1.320410e-01,-1.666623e-14,9.251740e-02,-3.320697e-01
3,"Missing, however, were details of all services...",1.778134e-01,-6.339399e-02,-2.418580e-01,-1.358887e-02,-2.859783e-01,-1.154145e-01,1.217411e-01,-5.700460e-14,5.442687e-01,6.684633e-01
4,"However, on Sunday, the Ministry of Health con...",4.933340e-01,5.457239e-01,-1.516730e-01,2.101868e-01,-9.260668e-02,-1.139804e-01,-5.998296e-02,6.331458e-15,-2.324718e-02,9.459912e-02
5,Regulated health professionals such as registe...,4.245538e-01,5.480899e-01,-1.479920e-01,2.085594e-01,6.538647e-02,-2.952794e-01,-8.397827e-02,1.158690e-14,3.125534e-02,-1.955789e-01
6,"In Ontario’s first lockdown last spring, physi...",3.080669e-01,7.774877e-02,-2.027386e-01,-9.654630e-02,-2.934623e-01,3.960914e-01,-5.022268e-01,6.734486e-14,-3.051934e-01,1.896720e-01
7,“We have to continue to communicate with clien...,2.297365e-01,-7.188601e-02,3.553376e-01,-3.225288e-01,3.313360e-01,-3.115954e-01,-2.421170e-01,2.020956e-14,6.209495e-02,1.830396e-01
8,"“It’s not like a haircut,” she said, adding th...",3.455652e-01,4.270226e-01,1.673223e-01,-7.361580e-02,1.607353e-01,2.307038e-01,2.743913e-01,-3.647758e-14,1.467646e-01,-1.413578e-01
9,"To Armstrong, massage has been as important as...",1.741465e-01,1.298045e-01,8.735880e-02,-4.016126e-01,3.514651e-01,4.545098e-01,-7.794496e-02,-1.689133e-14,4.568746e-01,-1.138566e-01


----------------------------------------------------------
[1.34140476 1.17559305 1.07635552 1.07191523 1.04617646 1.02389161
 1.00825    1.         0.99087419 0.98976066]
----------------------------------------------------------


,Terms,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
0,01,0.036643,-0.044690,-0.078024,-0.049069,0.029344,-0.012293,0.053926,-5.902458e-15,0.001758,-0.003533
1,12,0.036643,-0.044690,-0.078024,-0.049069,0.029344,-0.012293,0.053926,-5.869721e-15,0.001758,-0.003533
2,19,0.036643,-0.044690,-0.078024,-0.049069,0.029344,-0.012293,0.053926,-5.829481e-15,0.001758,-0.003533
3,able,0.172775,0.077704,0.166173,-0.003857,-0.124969,0.034739,0.154254,-1.110692e-14,-0.137313,0.109589
4,acted,0.025809,-0.059410,0.018332,0.201459,0.123673,0.124447,-0.021196,-1.720002e-15,0.034105,0.057151
...,...,...,...,...,...,...,...,...,...,...,...
178,via,0.110120,-0.102794,0.106428,0.077074,0.013866,-0.035120,-0.011916,1.792495e-15,-0.013192,0.019352
179,weekend,0.065728,-0.072308,0.009260,0.095635,0.037229,0.019172,-0.001119,9.196698e-17,-0.001974,-0.003710
180,whether,0.141052,-0.123791,0.012931,-0.042098,-0.010943,-0.132909,0.070457,-5.274450e-15,0.003789,-0.072353
181,working,0.038733,0.065103,-0.020970,0.029797,0.009807,-0.046237,-0.013561,1.883044e-15,0.005226,-0.032774


...

,Terms,topic7
122,people,0.321587
165,stress,0.287657
68,high,0.287657
140,right,0.287657
56,families,0.287657
...,...,...
170,time,-0.128392
14,around,-0.128392
80,last,-0.128392
10,among,-0.128392


,Terms,topic8
52,everyone,4.472136e-01
146,see,4.472136e-01
54,faces,4.472136e-01
57,fatigue,4.472136e-01
151,setting,4.472136e-01
...,...,...
73,included,-2.641267e-14
95,missing,-2.641267e-14
47,details,-2.641267e-14
154,shutdown,-2.820122e-14


,Terms,topic9
95,missing,0.252880
47,details,0.252880
73,included,0.252880
154,shutdown,0.222345
69,however,0.217423
...,...,...
56,families,-0.154684
68,high,-0.154684
165,stress,-0.154684
140,right,-0.154684


,Terms,topic10
95,missing,0.311283
73,included,0.311283
47,details,0.311283
69,however,0.285622
154,shutdown,0.268722
...,...,...
42,confusion,-0.114720
61,general,-0.114720
149,seems,-0.114720
130,public,-0.128789


----------------------------------------------------------


In [4]:
sentences = extractSummary(u, sigma, 5, corpus)
summary = '\n\n'.join(sentences)
print(summary)


“I see the fatigue setting in on everyone’s faces ...

Missing, however, were details of all services included in the shutdown.

However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”  A spokesperson said that “under lockdown, regulated health professionals, including massage therapists, will be able to operate.

Regulated health professionals such as registered massage therapists were not impacted and therefore not referenced.”  Working “in partnership with the chief medical officer of health and our local medical officers of health, we continue to closely monitor the evolving situation to advise if and when public health measures need to be adjusted,” the spokesperson also said.

The stress is so high … right now, (with) people not being able to be with their families.
